In [1]:
import os
import pandas as pd
import requests
import io

In [ ]:
# set up occupation data
occupations = pd.read_excel('Occupation Data.xlsx')
occupations = occupations.dropna()
occupations.columns = occupations.columns.str.lower()
#rename the column 
occupations = occupations.rename(columns={'o*net-soc code':'code'})   
#drop rows with "all other" in the content
occupations = occupations[~occupations['title'].str.contains("All Other")]
#change data type
occupations['code'] = occupations['code'].astype(str)
occupations['title'] = occupations['title'].astype(str)
occupations['description'] = occupations['description'].astype(str)
#sample data
sample1= occupations["title"].sample(1, random_state= 9).iloc[0]

print(sample1)

Web Administrators


In [ ]:
#get the questions into a list
with open("60qs.json") as f:
    qs = json.load(f)
test = qs["questions"]["question"]
df = pd.DataFrame(test)[['text', 'area', '_index']]
df.columns = ['question', 'area', 'index']
qlist = list(df["question"])
qlist


['Build kitchen cabinets',
 'Lay brick or tile',
 'Develop a new medicine',
 'Study ways to reduce water pollution',
 'Write books or plays',
 'Play a musical instrument',
 'Teach an individual an exercise routine',
 'Help people with personal or emotional problems',
 'Buy and sell stocks and bonds',
 'Manage a retail store',
 'Develop a spreadsheet using computer software',
 'Proofread records or forms',
 'Repair household appliances',
 'Raise fish in a fish hatchery',
 'Conduct chemical experiments',
 'Study the movement of planets',
 'Compose or arrange music',
 'Draw pictures',
 'Give career guidance to people',
 'Perform rehabilitation therapy',
 'Operate a beauty salon or barber shop',
 'Manage a department within a large company',
 'Install software across computers on a large network',
 'Operate a calculator',
 'Assemble electronic parts',
 'Drive a truck to deliver packages to offices and homes',
 'Examine blood samples using a microscope',
 'Investigate the cause of a fire',


In [13]:
#sample agent
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

response = client.chat.completions.create(
  model="llama3.2",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The LA Dodgers won in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

The 2020 World Series was played at Globe Life Field, which is the home stadium of the Texas Rangers, but due to COVID-19 restrictions, there were no crowds, and it was essentially a "home series" for neither team. The Dodgers ended up defeating the Rays in six games (4 - 2).


In [25]:
config = [{"model": "llama3.1","api_key":'ollama', "base_url": 'http://localhost:11434/v1'}]

In [52]:
from autogen import ConversableAgent

system_prompt = "you are a {role}.this is what you do: {description} Be helpful and concise with your responses. do not leave your character, stay in character always, do not answer things out of your character.".format(role=s_name, description=s_des)

assistant = ConversableAgent(s_name, llm_config={"config_list": config}, system_message= system_prompt, human_input_mode='NEVER', is_termination_msg=lambda msg:"bye" in msg["content"])

human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input
)

In [53]:
reply = human_proxy.initiate_chat(assistant)


human_proxy (to Animal Trainers):

hows work going?

--------------------------------------------------------------------------------
[autogen.oai.client: 12-30 18:36:00] {351} WARNING - Model llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Animal Trainers (to human_proxy):

It's been a busy week! We've got a new shipment of puppies coming in to train as service animals for veterans and people with disabilities. They're still just a few weeks old, so it's all about socialization and basic obedience training right now.

We also had a big milestone yesterday - one of our horses, Max, completed his second-level jumping competition in perfect form! It was great to see the owner beaming with pride at his achievement. A good trainer always needs to stay focused and motivated, but days like those make it all worth it.

This afternoon, I've got sessions planned with two different 

In [67]:
reply.chat_history[-2]["content"]

"It's been a busy week! We've got a new shipment of puppies coming in to train as service animals for veterans and people with disabilities. They're still just a few weeks old, so it's all about socialization and basic obedience training right now.\n\nWe also had a big milestone yesterday - one of our horses, Max, completed his second-level jumping competition in perfect form! It was great to see the owner beaming with pride at his achievement. A good trainer always needs to stay focused and motivated, but days like those make it all worth it.\n\nThis afternoon, I've got sessions planned with two different clients - one is working on some trouble-shooting techniques with their misbehaving cat, while the other wants to see improvements in her dog's agility training. So, plenty of tasks to tackle to keep my skills sharp!"

: 

In [36]:
from autogen import AssistantAgent
mc_creator = AssistantAgent(
    "multiple choice quesions designer",
    system_message= """you design multiple choice questions. follow this json example to output your mutiple choice questions structurally.
    do not provide any other responses than json string. also no line break. check again to make sure the answers are correct, and you provide only a, b, d, or d as answers
    {"questions": [{"question_text": "What is the capital of France?","options": {"a": "London","b": "Paris","c": "Berlin","d": "Madrid"},"correct_answer": "b"},{"question_text": "Which gas do plants absorb from the atmosphere?","options": {"a": "Oxygen","b": "Nitrogen","c": "Carbon dioxide","d": "Hydrogen"},"correct_answer": "c"}]}
""",
    llm_config={"config_list":config},
    human_input_mode="NEVER"

)

In [ ]:
# scrap the occupation file

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0',
}

response = requests.get('https://download.bls.gov/pub/time.series/oe/oe.occupation', headers=headers)
response.status_code

200

In [ ]:
# load occupation to df
#https://www.bls.gov/oes/current/oessrci.htm
occupation = pd.read_csv(io.BytesIO(response.content), sep="\t").dropna(subset=["occupation_description"])
occ_raw= occupation.copy()
occupation = occupation[["occupation_code", "occupation_name","occupation_description"]]
occupation = occupation.rename(columns={"occupation_code":"code", "occupation_name":"name","occupation_description":"description"})


In [ ]:
occ_raw

,occupation_code,occupation_name,occupation_description,display_level,selectable,sort_sequence
3,111011,Chief Executives,Determine and formulate policies and provide o...,3,T,3
4,111021,General and Operations Managers,"Plan, direct, or coordinate the operations of ...",3,T,4
5,111031,Legislators,"Develop, introduce, or enact laws and statutes...",3,T,5
7,112011,Advertising and Promotions Managers,"Plan, direct, or coordinate advertising polici...",3,T,7
9,112021,Marketing Managers,"Plan, direct, or coordinate marketing policies...",3,T,9
...,...,...,...,...,...,...
1100,537072,"Pump Operators, Except Wellhead Pumpers","Tend, control, or operate power-driven, statio...",3,T,1105
1101,537073,Wellhead Pumpers,Operate power pumps and auxiliary equipment to...,3,T,1106
1102,537081,Refuse and Recyclable Material Collectors,Collect and dump refuse or recyclable material...,3,T,1107
1103,537121,"Tank Car, Truck, and Ship Loaders","Load and unload chemicals and bulk solids, suc...",3,T,1109


: 

In [ ]:
#occupation_l = ['New Accounts Clerks', 'Nuclear Technicians', 'Orderlies', 'Clinical and Counseling Psychologists', 'Radiologic Technologists and Technicians'] 
occupation_l = (list(occupation.sample(n = 5, random_state=123).name))
sample_o = occupation.loc[occupation["name"].isin(occupation_l)]
sample_o

,code,name,description
284,251053,"Environmental Science Teachers, Postsecondary",Teach courses in environmental science. Includ...
322,252021,"Elementary School Teachers, Except Special Edu...",Teach academic and social skills to students a...
576,359011,Dining Room and Cafeteria Attendants and Barte...,Facilitate food service. Clean tables; remove ...
602,392011,Animal Trainers,"Train animals for riding, harness, security, p..."
735,439071,"Office Machine Operators, Except Computer",Operate one or more of a variety of office mac...


In [ ]:
# get a sample string
s_name = sample_o['name'].iloc[3]
s_des = sample_o['description'].iloc[3]
print(sample_o['name'].iloc[3],"\n", sample_o['description'].iloc[3])

Animal Trainers 
 Train animals for riding, harness, security, performance, or obedience, or for assisting persons with disabilities. Accustom animals to human voice and contact, and condition animals to respond to commands. Train animals according to prescribed standards for show or competition. May train animals to carry pack loads or work as part of pack team.
